In [5]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from config import api_key
import requests
import json

In [6]:
# Locate CSV files and convert to dataframes
STATE_DATA_PATH = "Resources/MN_2020_ADI_9 Digit Zip Code_v3.2.csv"
LOCAL_DATA_PATH = "Resources/Local_Repository_Data.csv"
PT_DATA_PATH = "Resources/Patient_Data.csv"

state_df = pd.read_csv(STATE_DATA_PATH)
local_df = pd.read_csv(LOCAL_DATA_PATH)
pt_df = pd.read_csv(PT_DATA_PATH)

C:\Users\jkaum\AppData\Local\Temp\ipykernel_27352\2531968484.py:6: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  state_df = pd.read_csv(STATE_DATA_PATH)


# DATA CLEANING
---
state_df cleaning

In [7]:
state_df.head()

,ZIP_4,GISJOIN,FIPS,ADI_NATRANK,ADI_STATERANK,TYPE
0,550010002,G27016300711022,2.716307e+11,12,1,
1,550010003,G27016300711022,2.716307e+11,12,1,
2,550010004,G27016300711022,2.716307e+11,12,1,
3,550010001,G27016300711022,2.716307e+11,12,1,
4,550010015,G27016300711022,2.716307e+11,12,1,


In [8]:
# Number of zip codes in state_df
print(f"There are {state_df.shape[0]} MN zip codes in state_df")

There are 1207314 MN zip codes in state_df


In [9]:
# Drop unwanted columns from state_df
clean_state_df = state_df.drop(["GISJOIN", "FIPS", "TYPE"], axis=1)
clean_state_df.head()

,ZIP_4,ADI_NATRANK,ADI_STATERANK
0,550010002,12,1
1,550010003,12,1
2,550010004,12,1
3,550010001,12,1
4,550010015,12,1


In [10]:
# Find bad data to drop
clean_state_df["ADI_STATERANK"].unique()

array(['1', '4', '3', '6', 'GQ-PH', '2', '5', '9', '8', '7', 'GQ', '10',
       'PH', nan, 8, 10, 7, 4, 9, 5, 6, 3, 1], dtype=object)

In [11]:
# Drop bad data
clean_state_df = clean_state_df.loc[(clean_state_df["ADI_STATERANK"] != "GQ") &
                                    (state_df["ADI_STATERANK"] != "PH") &
                                    (state_df["ADI_STATERANK"] != "GQ-PH")]
clean_state_df["ADI_STATERANK"].unique()

array(['1', '4', '3', '6', '2', '5', '9', '8', '7', '10', nan, 8, 10, 7,
       4, 9, 5, 6, 3, 1], dtype=object)

In [12]:
# Drop NaN data
clean_state_df = clean_state_df.dropna()

In [13]:
# Confirm that all bad data is dropped from ADI_STATERANK column
clean_state_df["ADI_STATERANK"].unique()

array(['1', '4', '3', '6', '2', '5', '9', '8', '7', '10', 8, 10, 7, 4, 9,
       5, 6, 3, 1], dtype=object)

In [14]:
# Confirm that all bad data is dropped from ADI_NATRANK column
clean_state_df["ADI_NATRANK"].unique()

array(['12', '11', '16', '35', '10', '39', '31', '50', '27', '42', '45',
       '22', '37', '74', '43', '62', '46', '40', '47', '34', '56', '67',
       '68', '69', '66', '70', '63', '41', '48', '38', '36', '57', '28',
       '25', '33', '29', '26', '30', '19', '55', '32', '49', '20', '18',
       '24', '23', '53', '54', '97', '14', '59', '15', '21', '17', '44',
       '73', '64', '81', '61', '58', '76', '60', '13', '9', '51', '8',
       '98', '5', '75', '86', '72', '65', '96', '84', '95', '77', '71',
       '82', '83', '6', '92', '52', '89', '80', '79', '78', '7', '99',
       '4', '100', '3', '2', '91', '85', '87', '93', '88', '94', '90', 60,
       62, 61, 96, 55, 63, 59, 39, 67, 44, 69, 42, 43, 74, 64, 79, 81, 70,
       77, 54, 68, 51, 46, 58, 52, 72, 82, 75, 80, 88, 56, 76, 45, 34, 90,
       65, 99, 40, 36, 78, 84, 83, 73, 86, 66, 31, 53, 50, 85],
      dtype=object)

In [15]:
# Reset the index after dropping rows
clean_state_df = clean_state_df.reset_index(drop=True)
clean_state_df.head()

,ZIP_4,ADI_NATRANK,ADI_STATERANK
0,550010002,12,1
1,550010003,12,1
2,550010004,12,1
3,550010001,12,1
4,550010015,12,1


In [16]:
# Print number of rows that were dropped during data cleaning
print(f"{state_df.shape[0] - clean_state_df.shape[0]} rows were dropped from state_df during data cleaning.")

29462 rows were dropped from state_df during data cleaning.


In [17]:
# Rename zip code column name to match the pt_data_df
clean_state_df = clean_state_df.rename(columns={"ZIP_4": "Zip Code (9-digit)"})
clean_state_df.head()

,Zip Code (9-digit),ADI_NATRANK,ADI_STATERANK
0,550010002,12,1
1,550010003,12,1
2,550010004,12,1
3,550010001,12,1
4,550010015,12,1


---
local_df cleaning

In [18]:
local_df.head()

,Local Repository ID,Zip Code (5-digit),Zip Code (9-digit),Established
0,101,55407,554074563,7/9/2021
1,102,56377,563771274,7/1/2021
2,103,55454,554544400,6/10/2021
3,104,55416,554162527,7/15/2022
4,105,56441,564411422,7/21/2022


In [19]:
# Print the number of repositories
print(f"RoundtableRx has {local_df.shape[0]} local repositories where patients can get their medications.")

RoundtableRx has 22 local repositories where patients can get their medications.


In [20]:
# Drop unwanted columns
clean_local_df = local_df.drop("Established", axis=1)
clean_local_df.head()

,Local Repository ID,Zip Code (5-digit),Zip Code (9-digit)
0,101,55407,554074563
1,102,56377,563771274
2,103,55454,554544400
3,104,55416,554162527
4,105,56441,564411422


In [21]:
# Check for bad data
clean_local_df["Zip Code (5-digit)"].unique()

array([55407, 56377, 55454, 55416, 56441, 56401, 56636, 56501, 55063,
       55807, 55987, 55404, 55104, 55409, 55054, 55303, 55805],
      dtype=int64)

In [22]:
# Check for bad data
clean_local_df["Zip Code (9-digit)"].unique()

array([554074563, 563771274, 554544400, 554162527, 564411422, 564013575,
       566368779, 565013503, 550636026, 564013054, 558072730, 554071131,
       559876435, 558072737, 554043074, 551044201, 554091304, 554044515,
       550545483, 554042923, 553035566, 558051901], dtype=int64)

----
pt_df cleaning

In [23]:
pt_df.head()

,Local Repository ID,Patient ID,Zip Code (5-digit),Zip Code (9-digit),Date Started or Renewed
0,113,2001,55987,559874346,1/30/2023
1,115,2002,55429,554292726,5/18/2021
2,115,2003,??,??,6/2/2021
3,115,2004,55407,554071135,8/20/2021
4,115,2005,55428,554282966,7/22/2022


In [24]:
# RoundtableRx serves 133 unique patients
print(f"RoundtableRx serves {pt_df.shape[0]} unique patients.")

RoundtableRx serves 133 unique patients.


In [25]:
# Drop unwanted columns
clean_pt_df = pt_df.drop("Date Started or Renewed", axis=1)
clean_pt_df.head()

,Local Repository ID,Patient ID,Zip Code (5-digit),Zip Code (9-digit)
0,113,2001,55987,559874346
1,115,2002,55429,554292726
2,115,2003,??,??
3,115,2004,55407,554071135
4,115,2005,55428,554282966


In [26]:
# Check for bad data
clean_pt_df["Zip Code (5-digit)"].unique()

array(['55987', '55429', '??', '55407', '55428', '55417', '55434',
       '55418', '55413', '55423', '56679', '55792', '55810', '55720',
       '56649', '55804', '55805', '56401', '55807', '55724', '55705',
       '55812', '55795', '56470', '55803', '56636', '55753', '56571',
       '56573', '55797', '55744', '55811', '56484', '55760', '55782',
       '56450', '56672', '56501', '55779', '56473', '56474', '56554',
       '56637', '56544', '56449', '55722', '56655', '55020', '55044',
       '55088', '55054', '55378', '55057', '55616', '55454', '55106',
       '55406', '55303', '55126', '55433', '55343', '55304', '55025',
       '55387', '55117', '55346', '55110', '55362', '55404', '56444',
       '56441', '55119'], dtype=object)

In [27]:
# Check for bad data
clean_pt_df["Zip Code (9-digit)"].unique()

array(['559874346', '554292726', '??', '554071135', '554282966',
       '554172816', '554195500', '554072203', '554184420', '554131969',
       '554234333', '566790093', '557922844', '558109576', '557201311',
       '566493402', '558041646', '557922200', '558052044', '564012737',
       '566492805', '558072658', '557248533', '557051441', '558051823',
       '558043301', '558121751', '557953076', '558052073', '564701387',
       '558072542', '558043061', '558039436', '566360179', '557530641',
       '565719409', '565739010', '566363071', '557978718', '557442177',
       '557248209', '558111236', '564842171', '557605894', '557820385',
       '564509694', '566724575', '565012449', '558032034', '557797006',
       '566360395', '565013203', '566362044', '564732139', '558102022',
       '557923527', '558041806', '558121720', '558115323', '564014402',
       '566362205', '564746164', '565549002', '566372091', '565012909',
       '565448547', '558071922', '558071908', '558031945', '564491107',

In [28]:
# Drop bad data
clean_pt_df = clean_pt_df.loc[(clean_pt_df["Zip Code (9-digit)"] != "??") & (clean_pt_df["Zip Code (5-digit)"] != "??")]
clean_pt_df.head()

,Local Repository ID,Patient ID,Zip Code (5-digit),Zip Code (9-digit)
0,113,2001,55987,559874346
1,115,2002,55429,554292726
3,115,2004,55407,554071135
4,115,2005,55428,554282966
5,115,2006,55417,554172816


In [29]:
# Reset the index after dropping rows
clean_pt_df = clean_pt_df.reset_index(drop=True)
clean_pt_df.head()

,Local Repository ID,Patient ID,Zip Code (5-digit),Zip Code (9-digit)
0,113,2001,55987,559874346
1,115,2002,55429,554292726
2,115,2004,55407,554071135
3,115,2005,55428,554282966
4,115,2006,55417,554172816


In [30]:
# Print number of pts lost during data cleaning
print(f"{pt_df.shape[0] - clean_pt_df.shape[0]} patients were dropped during data cleaning.")

4 patients were dropped during data cleaning.


# Data Merging

In [31]:
# Before merging clean_pt_df and clean_state_df, ensure that common column is of the same dtype
clean_pt_df['Zip Code (9-digit)'] = clean_pt_df['Zip Code (9-digit)'].astype(int)
clean_state_df['Zip Code (9-digit)'] = clean_state_df['Zip Code (9-digit)'].astype(int)

In [32]:
# Merge clean_pt_df and clean_state_df so that we can get the ADI rank of each patient
pt_adi_df = pd.merge(clean_pt_df,clean_state_df, how="inner", on=["Zip Code (9-digit)", "Zip Code (9-digit)"])
pt_adi_df.head()

,Local Repository ID,Patient ID,Zip Code (5-digit),Zip Code (9-digit),ADI_NATRANK,ADI_STATERANK
0,113,2001,55987,559874346,77,10
1,115,2002,55429,554292726,62,8
2,115,2004,55407,554071135,57,7
3,115,2005,55428,554282966,63,8
4,115,2006,55417,554172816,84,10


In [33]:
# Print number of pts lost during data merging
print(f"{clean_pt_df.shape[0] - pt_adi_df.shape[0]} patients were dropped during data merging.")

2 patients were dropped during data merging.


In [34]:
# Determine total number of patients with ADI ranks we can analyze
total_pts = pt_adi_df["Patient ID"].count()
print(f"ADI ranks are available for {total_pts} of RoundtableRx's patients.")

ADI ranks are available for 127 of RoundtableRx's patients.


In [35]:
# Before merging clean_local_df and clean_state_df, ensure that common column is of the same dtype
clean_local_df['Zip Code (9-digit)'] = clean_local_df['Zip Code (9-digit)'].astype(int)

In [36]:
# Merge clean_local_df and clean_state_df so that we can get the ADI rank of each local repository
local_adi_df = pd.merge(clean_local_df,clean_state_df, how="left", on=["Zip Code (9-digit)", "Zip Code (9-digit)"])
local_adi_df

,Local Repository ID,Zip Code (5-digit),Zip Code (9-digit),ADI_NATRANK,ADI_STATERANK
0,101,55407,554074563,44,5
1,102,56377,563771274,65,8
2,103,55454,554544400,NaN,NaN
3,104,55416,554162527,41,5
4,105,56441,564411422,58,7
5,106,56401,564013575,84,10
6,107,56636,566368779,90,10
7,108,56501,565013503,75,9
8,109,55063,550636026,63,8
9,110,56401,564013054,73,9


1.using geoapify (https://apidocs.geoapify.com/docs/geocoding/forward-geocoding/#geocode-zipcodes) add lat and lon columns as well get a county column to pt_adi_df and local_adi_df. get county column added to clean_state_df.
    - Ayan
    Travis

make the maps

2. get mean, mode for pt_adi_df ADI_STATERANK and ADI_NATRANK
    - Kokoe

make bar charts y = # of pts, x = ADIs core for pt_adi_df. state ranks only
    - Roli

3. pie chart ADI scores
    - Joanna

4. health access / health outcomes vs adi score per county

to create a new branch:     git checkout -b new-branch-name
to push to a branch:        git push origin new-branch-name

10 AM GROUP ON SATURDAY

2:30 GROUP ON SATURDAY


In [37]:
# Create a function to add latitude, longitude, and County columns to the state, local, and pt dataframes
def column_creation(dataframe):
    dataframe["LAT"]=""
    dataframe["LONG"] =""
    dataframe["County"]= ""  

In [38]:
# Run column_creation function on pt_adi_df
column_creation(pt_adi_df)
pt_adi_df.head()

,Local Repository ID,Patient ID,Zip Code (5-digit),Zip Code (9-digit),ADI_NATRANK,ADI_STATERANK,LAT,LONG,County
0,113,2001,55987,559874346,77,10,,,
1,115,2002,55429,554292726,62,8,,,
2,115,2004,55407,554071135,57,7,,,
3,115,2005,55428,554282966,63,8,,,
4,115,2006,55417,554172816,84,10,,,


In [39]:
# Run column_creation function on local_adi_df
column_creation(local_adi_df)
local_adi_df.head()

,Local Repository ID,Zip Code (5-digit),Zip Code (9-digit),ADI_NATRANK,ADI_STATERANK,LAT,LONG,County
0,101,55407,554074563,44,5,,,
1,102,56377,563771274,65,8,,,
2,103,55454,554544400,NaN,NaN,,,
3,104,55416,554162527,41,5,,,
4,105,56441,564411422,58,7,,,


In [40]:
# Add the Zip Code (5-digit) column to the clean_state_df
clean_state_df['Zip Code (5-digit)'] = clean_state_df['Zip Code (9-digit)'].apply(lambda x: str(x)[:5])
clean_state_df.head()

,Zip Code (9-digit),ADI_NATRANK,ADI_STATERANK,Zip Code (5-digit)
0,550010002,12,1,55001
1,550010003,12,1,55001
2,550010004,12,1,55001
3,550010001,12,1,55001
4,550010015,12,1,55001


In [41]:
# Create a dataframe that just has unique 5-digit MN zip codes and add empty columns for lat, long, and County
zip_county = {"Zip Code (5-digit)":clean_state_df["Zip Code (5-digit)"].unique(), "LAT": "", "LONG": "", "County": ""}
zip_county_df = pd.DataFrame(zip_county)
zip_county_df.head()

,Zip Code (5-digit),LAT,LONG,County
0,55001,,,
1,55003,,,
2,55005,,,
3,55006,,,
4,55007,,,


In [42]:
# Create a function to populate empty columns on state, local, and pt dataframes
def location_info(dataframe):
    for index, row in dataframe.iterrows():
        # Loop through zip codes
        zip_code = dataframe.loc[index, "Zip Code (5-digit)"]
        # define target url
        target_url = f"https://api.geoapify.com/v1/geocode/search?text={zip_code}&lang=en&limit=10&type=postcode&filter=countrycode:us&apiKey={api_key}"
        #get lat, lon, county
        latitude = dataframe.loc[index, "LAT"]
        longitude = dataframe.loc[index, "LONG"]
        county = dataframe.loc[index, "County"]
        # make API request
        response = requests.get(target_url).json()
        try:
            dataframe.loc[index, "LAT"] = response["features"][0]["properties"]["lat"]
            dataframe.loc[index, "LONG"] = response["features"][0]["properties"]["lon"]
            dataframe.loc[index, "County"] = response["features"][0]["properties"]["county"]
            print(f"{zip_code} found")
        except:
            dataframe.loc[index, "LAT"] = "Not Found"
            dataframe.loc[index, "LONG"] = "Not Found"
            dataframe.loc[index, "County"] = "Not Found"
            print(f"{zip_code} NOT found")


In [43]:
# # This function takes time to run so it was run just once, results saved as a csv and then commented out.
# # The path for the csv is "Output/local_repository_data_complete.csv"
# # csv later imported as local_adi_complete

# # Run location_info function on local_adi_df
# location_info(local_adi_df)
# local_adi_df

In [44]:
# # This function takes time to run so it was run just once, results saved as a csv and then commented out.
# # The path for the csv is "Output/patient_data_complete.csv"
# # csv later imported as pt_adi_complete

# # Run location_info function on pt_adi_df
# location_info(pt_adi_df)
# pt_adi_df

In [45]:
# # This function takes A LONG time to run so it was run just once, results saved as a csv and then commented out.
# # The path for the csv is "Output/local_repository_data_complete.csv"
# # csv later imported as state_complete

# # Run location_info function on zip_county_df
# location_info(zip_county_df)
# zip_county_df.head()

In [46]:
# # This function takes A LONG time to run so it was run just once, results saved as a csv and then commented out.
# # The path for the csv is "Output/local_repository_data_complete.csv"
# # csv later imported as state_complete

# # Drop any data where County was not found
# clean_zip_county_df = zip_county_df.loc[(zip_county_df["County"] != "NOT found")]
# clean_zip_county_df.head()

In [47]:
# # This function takes A LONG time to run so it was run just once, results saved as a csv and then commented out.
# # The path for the csv is "Output/local_repository_data_complete.csv"
# # csv later imported as state_complete

# # merge clean_zip_county_df with clean_state_df
# clean_state_df = pd.merge(clean_zip_county_df,clean_state_df, how="left", on=["Zip Code (5-digit)", "Zip Code (5-digit)"])
# clean_state_df.head()

In [48]:
# # This function was run once to save our final dataframes as csv files, then commented out so that we will not
# # mistakenly overwrite or append the csv files

# # Save completed dataframes to csv files. Once complete we will not have to run the location_info function
# # again, thus saving time whenever we need to restart the kernel
# local_adi_df.to_csv("Output/local_repository_data_complete.csv", index=False)
# pt_adi_df.to_csv("Output/patient_data_complete.csv", index=False)
# clean_state_df.to_csv("Output/ADI_data_complete.csv", index=False)

In [49]:
# Import CSV files and convert to dataframes
ADI_DATA_COMPLETE_PATH = "Output/local_repository_data_complete.csv"
LOCAL_REPOSITORY_DATA_COMPLETE_PATH = "Output/patient_data_complete.csv"
PATIENT_DATA_COMPLETE_PATH = "Output/ADI_data_complete.csv"

state_complete = pd.read_csv(ADI_DATA_COMPLETE_PATH)
local_adi_complete = pd.read_csv(LOCAL_REPOSITORY_DATA_COMPLETE_PATH)
pt_adi_complete = pd.read_csv(PATIENT_DATA_COMPLETE_PATH)

C:\Users\jkaum\AppData\Local\Temp\ipykernel_27352\1239156396.py:8: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  pt_adi_complete = pd.read_csv(PATIENT_DATA_COMPLETE_PATH)


In [50]:
state_complete.head()

,Local Repository ID,Zip Code (5-digit),Zip Code (9-digit),ADI_NATRANK,ADI_STATERANK,LAT,LONG,County
0,101,55407,554074563,44.0,5.0,44.938337,-93.253067,Hennepin County
1,102,56377,563771274,65.0,8.0,45.626310,-94.218518,Stearns County
2,103,55454,554544400,NaN,NaN,44.969891,-93.243023,Hennepin County
3,104,55416,554162527,41.0,5.0,44.947159,-93.342940,Hennepin County
4,105,56441,564411422,58.0,7.0,46.515680,-93.969317,Crow Wing County


In [51]:
local_adi_complete.head()

,Local Repository ID,Patient ID,Zip Code (5-digit),Zip Code (9-digit),ADI_NATRANK,ADI_STATERANK,LAT,LONG,County
0,113,2001,55987,559874346,77,10,44.046447,-91.657487,Winona County
1,115,2002,55429,554292726,62,8,45.061817,-93.340607,Hennepin County
2,115,2004,55407,554071135,57,7,44.938337,-93.253067,Hennepin County
3,115,2005,55428,554282966,63,8,45.052188,-93.378094,Hennepin County
4,115,2006,55417,554172816,84,10,44.907389,-93.231786,Hennepin County


In [52]:
pt_adi_complete.head()

,Zip Code (5-digit),LAT,LONG,County,Zip Code (9-digit),ADI_NATRANK,ADI_STATERANK
0,55001,44.913206689,-92.805283599,Washington County,550010002,12,1
1,55001,44.913206689,-92.805283599,Washington County,550010003,12,1
2,55001,44.913206689,-92.805283599,Washington County,550010004,12,1
3,55001,44.913206689,-92.805283599,Washington County,550010001,12,1
4,55001,44.913206689,-92.805283599,Washington County,550010015,12,1
